In [1]:
source("../common-ffpe-snvf/R/eval.R")
library(ggplot2)
library(scales)
library(dplyr)
library(patchwork)

Loading required package: filenamer


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
get_ct_count <- function(ffpe_annot, ff_annot, dataset_id, case_id_col, sample_id_col = "sample_name") {
	
	ct_count <- list()

	for (i in seq_len(nrow(ffpe_annot))){
		
		case_id <- ffpe_annot[i, case_id_col]
		ffpe_sample_name <- ffpe_annot[i, sample_id_col]

		message(sprintf("processing %s. %s", i, ffpe_sample_name))

		matched_ff_annot <- ff_annot[ff_annot[[case_id_col]] == case_id, ]
		ff_sample_names <- matched_ff_annot[[sample_id_col]]
		
		ffpe_snv <- read_vcf(file.path(vcf_dir, ffpe_sample_name, sprintf("%s.vcf", ffpe_sample_name)), columns = c("chrom", "pos", "ref", "alt"))
		# Filter C:G>T:A variants
		ffpe_snv_ct <- ct_filter(ffpe_snv)

		gt_paths <- file.path(vcf_dir, ff_sample_names, sprintf("%s.vcf", ff_sample_names))
		gt_snv <- snv_union(gt_paths)
		# Filter C:G>T:A variants
		gt_snv_ct <- ct_filter(gt_snv)

		## Count C>T variants
		n_ct_ffpe <- nrow(ffpe_snv_ct)
		n_ct_gt <- nrow(gt_snv_ct)

		sample_ct_count <- data.frame(
			case_id = case_id,
			ffpe_sample_name = ffpe_sample_name,
			matched_ff_sample_name = paste0(ff_sample_names, collapse = ";"),
			n_ct_ffpe = n_ct_ffpe,
			n_ct_gt = n_ct_gt
		)

		ct_count[[i]] <- sample_ct_count

	}

	all_ct_count <- do.call(rbind, ct_count)
	## Wide to Long
	ct_count_long <- reshape(all_ct_count, idvar = c("ffpe_sample_name", "matched_ff_sample_name"), direction = "long", v.names = "n_ct", varying = c("n_ct_ffpe", "n_ct_gt"), timevar = "sample_type", times = c("FFPE", "Ground Truth"))
	row.names(ct_count_long) <- NULL
	ct_count_long$dataset_id <- dataset_id

	ct_count_long
}


In [3]:
make_bar_chart_paired <- function(df){
	ggplot(data = df, aes(x = ffpe_sample_name, y = n_ct, fill = sample_type)) +
	facet_wrap(~dataset_id, scales = "free") +
	geom_bar(stat="identity", position=position_dodge()) +
	scale_y_continuous(labels = label_number()) +
	labs(
		x = "Samples", 
		y = "C>T Mutation Count",
		# fill = "Sample Type",
		# title = "C:G>T:A SNV Count"
	) +
	# theme_minimal() +
	theme(
		plot.title = element_text(hjust = 0.5, face = "bold", size = 12, margin = margin(t = 10, b = 10)),
		plot.subtitle = element_text(hjust = 0.5),
		plot.caption = element_text(hjust = 0),
		legend.position = "bottom",
		legend.title = element_blank(), # element_text(size = 10)
		legend.text = element_text(size = 8),
		legend.key.size = unit(0.5, "cm"),
		axis.text.x = element_blank(),
		axis.ticks.x = element_blank(),
		panel.background = element_rect(fill = "white", color = NA)
		# axis.title.x = element_text(size = 10),
		# axis.title.y = element_text(size = 10),
		
	)
}

In [4]:
get_ct_count_case_based <- function(ffpe_annot, ff_annot, dataset_id, case_id_col, sample_id_col = "sample_name"){

	cases <- unique(ffpe_annot[[case_id_col]])
	ct_count <- list()

	for (i in seq_len(length(cases))){

		case_id <- cases[i]
		ffpe_samples <- ffpe_annot[ffpe_annot[[case_id_col]] == case_id, ]
		ffpe_sample_names <- ffpe_samples[[sample_id_col]]

		message(sprintf("Processing Case: %s", case_id))

		ffpe_ct_count <- list()

		for (j in seq_len(length(ffpe_sample_names))){
			ffpe_snv <- read_vcf(file.path(vcf_dir, ffpe_sample_names[j], sprintf("%s.vcf", ffpe_sample_names[j])), columns = c("chrom", "pos", "ref", "alt"))
			ffpe_snv_ct <- ct_filter(ffpe_snv)

			message(sprintf("	FFPE Sample: %s", ffpe_sample_names[j]))
			
			ffpe_sample_ct_count <- data.frame(
				case_id = case_id,
				sample_name = ffpe_sample_names[j],
				sample_type = sprintf("FFPE"),
				n_ct = nrow(ffpe_snv_ct)
			)
			
			ffpe_ct_count[[j]] <- ffpe_sample_ct_count
		}

		ffpe_ct_count <- do.call(rbind, ffpe_ct_count)

		ff_samples <- ff_annot[ff_annot[[case_id_col]] == case_id, ]
		ff_sample_names <- ff_samples[[sample_id_col]]

		ff_ct_count <- list()

		for (j in seq_len(length(ff_sample_names))){
			ff_snv <- read_vcf(file.path(vcf_dir, ff_sample_names[j], sprintf("%s.vcf", ff_sample_names[j])), columns = c("chrom", "pos", "ref", "alt"))
			ff_snv_ct <- ct_filter(ff_snv)

			message(sprintf("	Fresh Frozen Sample: %s", ff_sample_names[j]))
			
			ff_sample_ct_count <- data.frame(
				case_id = case_id,
				sample_name = ff_sample_names[j],
				sample_type = sprintf("Fresh Frozen"),
				n_ct = nrow(ff_snv_ct)
			)
			
			ff_ct_count[[j]] <- ff_sample_ct_count
		}

		ff_ct_count <- do.call(rbind, ff_ct_count)

		case_ct_count <- rbind(ffpe_ct_count, ff_ct_count)
		ct_count[[i]] <- case_ct_count
	}

	ct_count <- do.call(rbind, ct_count)
	ct_count$dataset_id <- dataset_id
	ct_count
}

In [5]:
make_dot_plot <- function(df){
	ggplot(df, aes(x = case_id, y = n_ct, color = sample_type)) +
	geom_point(alpha = 0.3, size = 3) +
	facet_wrap(~dataset_id) +
	scale_y_continuous(labels = label_number()) +
	labs(
		x = "Cases", 
		y = "C>T count",
		color = "Sample Type",
		# title = "C:G>T:A SNV Count"
	) +
	# theme_minimal() +
	theme(
		legend.position = "bottom",
		legend.title = element_blank(), # element_text(size = 10)
		axis.text.x = element_text(angle = 45, hjust=1.1), # element_blank(),
		panel.background = element_rect(fill = "white", color = NA)
		# plot.title = element_text(hjust = 0.5, face = "bold", size = 12, margin = margin(t = 10, b = 10)),
		# plot.subtitle = element_text(hjust = 0.5),
		# plot.caption = element_text(hjust = 0),
		# axis.title.x = element_text(size = 10),
		# axis.title.y = element_text(size = 10),
		# axis.ticks.x = element_blank(),
		# legend.text = element_text(size = 8),
		# legend.key.size = unit(0.5, "cm"),
	)
}

In [6]:
make_bar_chart <- function(df){
	ggplot(data = df, aes(x = case_id, y = n_ct, fill = sample_type, group = sample_name)) +
	facet_wrap(~dataset_id, scales = "free") +
	geom_bar(stat="identity", position=position_dodge(), color = "black") +
	scale_y_continuous(labels = label_number()) +
	labs(
		x = "Case", 
		y = "C>T count",
		# fill = "Sample Type",
		# title = "C:G>T:A SNV Count"
	) +
	# theme_minimal() +
	theme(
		plot.title = element_text(hjust = 0.5, face = "bold", size = 12, margin = margin(t = 10, b = 10)),
		plot.subtitle = element_text(hjust = 0.5),
		plot.caption = element_text(hjust = 0),
		legend.position = "bottom",
		legend.title = element_blank(), # element_text(size = 10)
		legend.text = element_text(size = 8),
		legend.key.size = unit(0.5, "cm"),
		axis.text.x = element_text(angle = 45, vjust = 1, hjust=1.2),
		# axis.ticks.x = element_blank(),
		panel.background = element_rect(fill = "white", color = NA)
		# axis.title.x = element_text(size = 10),
		# axis.title.y = element_text(size = 10),
		
	) + 
	scale_fill_brewer(palette = "Paired")
	
}

In [7]:
w= 5; h = 3
options(repr.plot.width = w, repr.plot.height = h)


In [8]:
## Dataset specific variables
dataset_id <- "SRP044740"
vcf_dir <- "../vcf/SRP044740/vcf_filtered_pass_orientation"
lookup_table <- read.delim("../annot/SRP044740/sample-info_stage1.tsv")
ffpe_annot <- lookup_table[lookup_table$sample_type == "FFPE", ]
ff_annot <- lookup_table[lookup_table$sample_type == "FROZ", ]
sample_id_col <- "sample_name"
match_id_col <- "sample_number"


In [9]:
srp_ct_count <- get_ct_count_case_based(ffpe_annot, ff_annot, "ENA SRP044740", match_id_col, sample_id_col)
srp_ct_count$case_id <- sprintf("%02d", srp_ct_count$case_id)
qwrite(srp_ct_count, "SRP044740/SRP044740_ct_count.tsv")

srp_bar_chart <- make_bar_chart(srp_ct_count)
srp_dot_plot <- make_dot_plot(srp_ct_count)

qwrite(srp_bar_chart, "SRP044740/SRP044740_ct_count_bar_chart.rds")
qwrite(srp_dot_plot, "SRP044740/SRP044740_ct_count_dot_plot.rds")

qdraw(srp_bar_chart, "SRP044740/SRP044740_ct_count_bar_chart.pdf", width = 5, height = 3)
qdraw(srp_dot_plot, "SRP044740/SRP044740_ct_count_dot_plot.pdf", width = 5, height = 3)

Processing Case: 1



	FFPE Sample: BGI-FFPE1_SRR1523243

	FFPE Sample: BGI-FFPE1_SRR1523257

	FFPE Sample: BGI-FFPE1_SRR1523263

	Fresh Frozen Sample: BGI-FROZ1_SRR1523230

Processing Case: 2

	FFPE Sample: BGI-FFPE2_SRR1523244

	FFPE Sample: BGI-FFPE2_SRR1523264

	Fresh Frozen Sample: BGI-FROZ2_SRR1523231

Processing Case: 3

	FFPE Sample: BGI-FFPE3_SRR1523251

	FFPE Sample: BGI-FFPE3_SRR1523269

	Fresh Frozen Sample: BGI-FROZ3_SRR1523240

Processing Case: 4

	FFPE Sample: BGI-FFPE4_SRR1523253

	FFPE Sample: BGI-FFPE4_SRR1523268

	Fresh Frozen Sample: BGI-FROZ4_SRR1523239

Processing Case: 5

	FFPE Sample: BGI-FFPE5_SRR1523252

	FFPE Sample: BGI-FFPE5_SRR1523261

	Fresh Frozen Sample: BGI-FROZ5_SRR1523238

Processing Case: 6

	FFPE Sample: BGI-FFPE6_SRR1523256

	FFPE Sample: BGI-FFPE6_SRR1523254

	Fresh Frozen Sample: BGI-FROZ6_SRR1523242

Processing Case: 7

	FFPE Sample: BGI-FFPE7_SRR1523247

	FFPE Sample: BGI-FFPE7_SRR1523266

	Fresh Frozen Sample: BGI-FROZ7_SRR1523234

Processing Case: 8

	FFPE Sample

In [10]:
## Dataset specific variables
dataset <- "PRJEB8754"
vcf_dir <- "../vcf/PRJEB8754/vcf_pass-orient-pos-sb_ad_filtered"
lookup_table <- read.delim("../annot/PRJEB8754/sample-info_matched-ff-ffpe_on-pat-id-sample-type.tsv")
ffpe_annot <- lookup_table[lookup_table$preservation == "FFPE", ]
ff_annot <- lookup_table[lookup_table$preservation == "Frozen", ]
sample_id_col <- "sample_name"
case_id_col <- "inferred_id"

In [12]:
betge_ct_count <- get_ct_count_case_based(ffpe_annot, ff_annot, "betge15", case_id_col, sample_id_col)
qwrite(betge_ct_count, "PRJEB8754/PRJEB8754_ct_count.tsv")

betge_bar_chart <- make_bar_chart(betge_ct_count)
betge_dot_plot <- make_dot_plot(betge_ct_count)

qwrite(betge_bar_chart, "PRJEB8754/PRJEB8754_ct_count_bar_chart.rds")
qwrite(betge_dot_plot, "PRJEB8754/PRJEB8754_ct_count_dot_plot.rds")

qdraw(betge_bar_chart, "PRJEB8754/PRJEB8754_ct_count_bar_chart.pdf", width = 5, height =  3)
qdraw(betge_dot_plot, "PRJEB8754/PRJEB8754_ct_count_dot_plot.pdf", width = 5, height =  3)

Processing Case: Pat01

	FFPE Sample: Pat01_Meta_FFPE_ERR791893

	Fresh Frozen Sample: Pat01_Meta_Frozen_ERR791883



Processing Case: Pat03

	FFPE Sample: Pat03_Meta_FFPE_ERR791895

	Fresh Frozen Sample: Pat03_Meta_Frozen_ERR791884

Processing Case: Pat04

	FFPE Sample: Pat04_Meta_FFPE_ERR791896

	Fresh Frozen Sample: Pat04_Meta_Frozen_ERR791891

Processing Case: Pat08

	FFPE Sample: Pat08_Meta_FFPE_ERR791901

	Fresh Frozen Sample: Pat08_Meta_Frozen_ERR791886

Processing Case: Pat09

	FFPE Sample: Pat09_Meta_FFPE_ERR791902

	Fresh Frozen Sample: Pat09_Meta_Frozen_ERR791887

Processing Case: Pat10

	FFPE Sample: Pat10_Meta_FFPE_ERR791903

	Fresh Frozen Sample: Pat10_Meta_Frozen_ERR791888

Processing Case: Pat11

	FFPE Sample: Pat11_Meta_FFPE_ERR791905

	Fresh Frozen Sample: Pat11_Meta_Frozen_ERR791889

Processing Case: Pat12

	FFPE Sample: Pat12_Meta_FFPE_ERR791907

	Fresh Frozen Sample: Pat12_Meta_Frozen_ERR791911

Processing Case: Pat13

	FFPE Sample: Pat13_Meta_FFPE_ERR791908

	Fresh Frozen Sample: Pat13_Meta_Frozen_ERR791890

Processing Case: Pat14

	FFPE Sample: Pat14_Meta_FFPE_ERR791909

	Fresh 